# Exploratory Analysis

In [2]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
import os

In [3]:
from olist.data import Olist
data = Olist().get_data()

/home/branchedelac/code/branchedelac/04-Decision-Science/01-Project-Setup/data-context-and-setup/olist/../data/csv/
['olist_sellers_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_order_items_dataset.csv', 'olist_customers_dataset.csv', 'olist_orders_dataset.csv', 'olist_order_payments_dataset.csv', 'product_category_name_translation.csv', 'olist_products_dataset.csv', 'olist_geolocation_dataset.csv']
['sellers', 'order_reviews', 'order_items', 'customers', 'orders', 'order_payments', 'product_category_name_translation', 'products', 'geolocation']


Each transaction on the Olist ecommerce platform is characterized by:
- a `customer_id`, that would buy...
- various`product_id`...
- to a `seller_id`...
- and leaves a `review_id`...
- all this belonging to an `order_id`

## 1 - Run an automated exploratory analysis with [ydata-profiling](https://github.com/ydataai/ydata-profiling)

In [4]:
# First, let's install the ydata-profiling package
! pip install --quiet pandas==1.4.4 ydata_profiling==4.1.2

In [5]:
# Then create a "reports" directory
!mkdir reports

In [6]:
# let's import ProfileReport from ydata_profiling
from ydata_profiling import ProfileReport
# choose which datasets to profile
datasets_to_profile = ['orders', 'products', 'sellers',
                  'customers', 'order_reviews',
                  'order_items']

👉 Create and save one `html report` per dataset to profile 

⏳ (It usually takes a few minutes)

In [7]:
for d in datasets_to_profile:
    print('exporting: '+ d)
    profile = ProfileReport(data[d], title = d)
    profile.to_file(f"reports/{d}_report.html")

exporting: orders


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

...

exporting: order_items


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## 2 - Investigate the cardinalities of your various DataFrames

❓ **How many unique `orders`, `reviews`, `sellers`, `products` and `customer` is there ?**  
(You can use ydata-profiling or pandas methods on your notebook if you prefer)

In [47]:
for key, df in data.items():
    print(key)
    n_rows =  df.shape[0]
    n_unique = df.drop_duplicates().shape[0]
    if n_rows != n_unique:
        print("Uhoh!")
    print("Number of rows:", n_rows)
    print("Number of unique rows:", n_unique)
    print("Number of unique IDs:", df.iloc[:,0].nunique(), "\n")
    print(df.columns)
    #display(df.info())
    #display(df.head(1))


sellers
Number of rows: 3095
Number of unique rows: 3095
Number of unique IDs: 3095 

Index(['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'], dtype='object')
order_reviews
Number of rows: 99224
Number of unique rows: 99224
Number of unique IDs: 98410 

Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')
order_items
Number of rows: 112650
Number of unique rows: 112650
Number of unique IDs: 98666 

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')
customers
Number of rows: 99441
Number of unique rows: 99441
Number of unique IDs: 99441 

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')
orders
Number of rows: 99441
Number of unique rows: 99441
Number of unique

❓ **How many reviews is there per order? Do we have reviews for all orders ?**
<details>
    <summary markdown='span'>Hints</summary>

This info is not directly accessible in your individual csv. You'll need to proceed to merge
</details>

In [31]:
orders_and_reviews = data["order_reviews"].merge(data["orders"], how="left", on="order_id")
orders_and_reviews.info()
orders_and_reviews.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99224 entries, 0 to 99223
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   review_id                      99224 non-null  object
 1   order_id                       99224 non-null  object
 2   review_score                   99224 non-null  int64 
 3   review_comment_title           11568 non-null  object
 4   review_comment_message         40977 non-null  object
 5   review_creation_date           99224 non-null  object
 6   review_answer_timestamp        99224 non-null  object
 7   customer_id                    99224 non-null  object
 8   order_status                   99224 non-null  object
 9   order_purchase_timestamp       99224 non-null  object
 10  order_approved_at              99068 non-null  object
 11  order_delivered_carrier_date   97468 non-null  object
 12  order_delivered_customer_date  96359 non-null  object
 13  o

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00


🧪 **Test your code below**

Store the number of orders with missing reviews as `int` in a variable named `n_missing_reviews`

In [45]:
all_orders_and_reviews = data["order_reviews"].merge(data["orders"], how="outer", on="order_id")
orders_wo_reviews = all_orders_and_reviews[all_orders_and_reviews["review_id"].isnull()]
n_missing_reviews = orders_wo_reviews.shape[0]

display(orders_wo_reviews.head(1))
print(n_missing_reviews
      )

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
99224,NaN,403b97836b0c04a622354cf531062e5f,NaN,NaN,NaN,NaN,NaN,738b086814c6fcc74b8cc583f8516ee3,delivered,2018-01-02 19:00:43,2018-01-02 19:09:04,2018-01-03 18:19:09,2018-01-20 01:38:59,2018-02-06 00:00:00


768


In [46]:
from nbresult import ChallengeResult

result = ChallengeResult('exploratory',
    n=n_missing_reviews
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/branchedelac/.pyenv/versions/3.10.6/envs/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/branchedelac/code/branchedelac/04-Decision-Science/01-Project-Setup/data-exploratory-analysis/tests
plugins: anyio-3.6.2, asyncio-0.19.0, typeguard-2.13.3
asyncio: mode=strict
collecting ... collected 1 item

test_exploratory.py::TestExploratory::test_n_missing_reviews PASSED      [100%]

============================== 1 passed in 0.04s ===============================


💯 You can commit your code:

git add tests/exploratory.pickle

git commit -m 'Completed exploratory step'

git push origin master

